# CSE6242 Project
### Main Idea:
1.Given **business X categories** matrix, more generally categorize each business using **KNN**.

2.Use users' reviews on business to determine their preference by:
    $\rightarrow$Calculate each user's average normalized ratings for each category;
    $\rightarrow$Create a final **user X categories** matrix with ratings.

In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
b_cat = pd.read_csv('business_category.csv')

### Part1. Data Cleaning

In [2]:
#drop the column 'Restaurants'
b_cat.drop(['Restaurants','Food'],1, inplace=True)
b_copy = b_cat.copy()

#convert 'T' 'F' to 0,1
b_copy[b_copy.columns[2:]] = b_copy[b_copy.columns[2:]].astype(float)
#determine the freq table
b_cat_freq = (b_copy==1).sum()[2:]

In [3]:
def sparse_matrix (base_type=float):
    from collections import defaultdict
    return defaultdict (lambda: defaultdict (base_type))

In [4]:
#only consider the columns that has freq>500 as our largest groups
large_col = b_cat_freq[b_cat_freq>500].keys().tolist()

#create the sparse matrix based on the raw matrix b_copy above
B = sp.csr_matrix(b_copy[b_copy.columns[2:]].values)

### Part2. Generalize Categories By KNN

In [5]:
from collections import defaultdict
nrow = B.shape[0]
ncol = B.shape[1]
#create a dictionary to store {col[i]:(large_col[j],similarity degree)}
degree = defaultdict(float)
b_T = b_copy[b_copy.columns[2:]].transpose()

for i in range(ncol):
    degree_i = 0.0 #initialize
    for j in range(len(large_col)):
        #for each category i, find the dot product value with main category j
        bj = sp.csr_matrix(b_T.loc[large_col[j]:large_col[j]])
        degree_j = bj*B[:,i]
        
        #store the value only if it is greater than all previuos
        #-->get max degree
        if (degree_j.nnz!=0):   
            if (degree_j.data[0]>degree_i):
                degree[b_copy.columns[i+2]]=(large_col[j],degree_j.data[0])
                degree_i = degree_j.data[0]
                
    #check if there is no degree found, assign it 0.0 
    if (degree_i == 0.0):
        degree[b_copy.columns[i+2]]=(b_copy.columns[i+2],0.0)

Store the **business X (orig_cat,general_cat)** matrix as b as record.

In [46]:
#test only
b_cat.columns[50:100]
#degree['Chicken Shop']
#large_col

['American (New)',
 'American (Traditional)',
 'Bars',
 'Breakfast & Brunch',
 'Burgers',
 'Cafes',
 'Chicken Wings',
 'Chinese',
 'Fast Food',
 'Italian',
 'Japanese',
 'Mexican',
 'Nightlife',
 'Pizza',
 'Sandwiches',
 'Seafood',
 'Sports Bars',
 'Steakhouses',
 'Sushi Bars']

In [6]:
b = b_copy.copy()

#Replace the category name in following form: 
    # (original category, generalized category)
for i in range(ncol):
    b.rename(columns={b.columns[i+2]:b.columns[i+2]+','+degree[b.columns[i+2]][0]},inplace=True)

#drop the categories that has degree == 0 (not belongs to any large group)
for i in b.columns[2:]:
    if i.split(',')[1] not in large_col:
        b.drop(i,1,inplace=True)
    

Create the **business X general_category** sparese matrix for further steps.

In [7]:
C = sparse_matrix()

#for each business, get it's generalized category if exists
for i in range(nrow):
    for j in range(2,b.shape[1]):
        if (b[b.columns[j]].iloc[i]==1):
            j_cat = b.columns[j].split(',')[1]
            C[b[b.columns[1]].iloc[i]][j_cat] =+ 1
        

In [8]:
C['__etvGuL2dh_a1LOT0gNYQ']

defaultdict(float, {'American (Traditional)': 1, 'Bars': 1, 'Nightlife': 1})

### Part3. Integrate "Reviews" with Big Categories
Given **"reviews.csv"** file and business X categories matrix derived above, determine each user's preference by calculating the average normalized ratings for each category.

In [ ]:
reviews = pd.read_csv('merged_all_reviews.csv')

In [ ]:
#add one column 'Category' with each business's list of large category
reviews['Category'] = [C[x] for x in reviews['BusinessID']]

#extract each user's collection of reviews with ratings
#First, create a user*category matrix
U_Re_Cat = defaultdict(list)


row = reviews.shape[0] 
users = reviews['UserID'].unique()

for i in range(row):
    for user in users:
        if (reviews['UserID'].iloc[i]==user):
            U_Re_Cat[user].append(reviews['Category'].iloc[i])
        
        

In [ ]:
#create a user*category matrix
U_C = sparse_matrix()
for user in users:
    for 
    U_C[user][]